In [ ]:
from sklearn import linear_model
import pandas as pd
print('hello')
df=pd.read_excel('recup_data.xlsx')
r, c= df.shape
print(r,c)

In [ ]:
import matplotlib.pyplot as plt
xcols=df.columns[:-1]
print(xcols)
ycols=df.columns[-1]
print(ycols)
sel=df['ds']>0.03
X=df[sel][xcols].values
y=df[sel][ycols].values
print(X.shape)
reg = linear_model.LinearRegression()
model=reg.fit(X, y)
print('model fit over')
print(model.score(X,y))
print(model.intercept_)
print(model.coef_)
yhat=model.predict(X)
fig = plt.figure()
ax = fig.add_subplot()
plt.scatter(yhat, y)
# square plot
ax.set_aspect('equal', adjustable='box')


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
reg = linear_model.LinearRegression()
u=X['init_velocity']
v=X['final_velocity']
u=np.array(u)
v=np.array(v)
inp=np.multiply(u,u) - np.multiply(v,v)
inp=inp.reshape(-1,1)
model=reg.fit(inp,y)
print('model fit over')
print(model.score(inp,y))
print(model.intercept_)
print(model.coef_)
yhat=model.predict(inp)
fig = plt.figure()
ax = fig.add_subplot()
plt.scatter(yhat, y)
# square plot
ax.set_aspect('equal', adjustable='box')

In [ ]:
import numpy as np
x=np.array([1,2,3,4])
y=np.array([1,2,3,4])
print(x*y)

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

df_sine=pd.read_excel('sinedata.xlsx')
print(df_sine.shape)
print(df_sine.columns)
#print(df_sine)
fig, ax=plt.subplots()
ax.plot(df_sine['x'], df_sine['sinx_measured'], color='blue', linestyle='None', marker='o')
ax.plot(df_sine['x'], df_sine['sinx_exact'], color='red', linestyle='solid')

In [ ]:
#linear regression on sine data
from sklearn.linear_model import LinearRegression, Ridge,  Lasso, ElasticNet
from sklearn.metrics import mean_squared_error  # we will use MSE for evaluation
x=df_sine['x'].values
y_meas=df_sine['sinx_measured'].values
y=df_sine['sinx_exact'].values
reg = LinearRegression()
x=x.reshape(-1,1)
model=reg.fit(x,y_meas)
print('model fit over')
print(model.score(x,y_meas))
print(model.intercept_)
print(model.coef_)
yhat=model.predict(x)
fig, ax=plt.subplots()
ax.plot(x, y_meas, linestyle='None', color='blue', marker='o' )
ax.plot(x, yhat, color='red')

In [ ]:
#functions for polynomial fit to the sine data
import numpy as np
from sklearn.linear_model import LinearRegression, Ridge,  Lasso, ElasticNet
from sklearn.metrics import mean_squared_error  # we will use MSE for evaluation
def prep_inp(x, deg=1): 
    lenx=len(x)
    inp=np.zeros((lenx, deg))
    #print('inp.shape=', inp.shape)
    r, c = x.shape
    inp[:,0]=x.reshape(r, )
    #print(inp)
    for n in range(1, deg):
        z=inp[:,n-1].reshape(r,)
        z=z*x.reshape(r,)
        inp[:,n]=z
    return inp

def fit_poly(inp, y_meas, prm_alpha=0, method = 'Linear Regression'):
    eps=1e-7
    if (method=='Linear Regression') or (abs(prm_alpha)<eps):
        print('Linear Regression')
        reg = LinearRegression()
    elif method=='Ridge':
        print('Ridge Regularization')
        reg = Ridge(alpha=prm_alpha)
    elif method=='Lasso':
        print('Lasso Regularization')
        reg = Lasso(alpha=prm_alpha)
    else: 
        pass    
    model=reg.fit(inp,y_meas)
    return model    

In [ ]:
#polynomial fit to the sine data
import pandas as pd
import numpy as np
df_sine=pd.read_excel('sinedata.xlsx')
x=df_sine['x'].values
y_meas=df_sine['sinx_measured'].values
y=df_sine['sinx_exact'].values
inp=prep_inp(x, 12)
model=fit_poly(inp, y_meas, prm_alpha=1e-6, method='Linear Regression')
yhat=model.predict(inp)
eps=np.sum((yhat-y_meas)*(yhat-y_meas))
print('eps=', eps)
fig, ax=plt.subplots()
ax.plot(x, y_meas, linestyle='None', color='blue', marker='o' )
ax.plot(x, yhat, color='red')
ax.plot(x, y, linestyle='dotted', color='blue')

In [ ]:
#gradient computation 
def grad(inp_z, y_meas, W, alpha=0):
    #alpha>0 --> ridge regression
    #alpha=0 --> no regularization - simple linear regression
    yhat=np.matmul(inp_z,W).reshape(-1,1)
    gradient = np.matmul(inp_z.T,(yhat-y_meas))+alpha*W
    return yhat, gradient

In [ ]:
#regression using gradient descent
import pandas as pd
import numpy as np
df_sine=pd.read_excel('sinedata.xlsx')
x=df_sine['x'].values
y_meas=df_sine['sinx_measured'].values
y=df_sine['sinx_exact'].values
inp=prep_inp(x, 12)
y_meas=y_meas.reshape(-1,1)
ones_vec = np.ones((inp.shape[0], 1), dtype=inp.dtype)
inp_z=np.hstack([ones_vec, inp])
#W=np.random.random((inp.shape[1]+1,1))
W=2000*(np.random.random((inp.shape[1]+1,1)) - 0.5)
yhat, gradient = grad(inp_z, y_meas, W, alpha=1e-2)
print('gradient.shape=', gradient.shape)
eps=10.0
eta=0.01
iter=0
#while (eps>2.5):  
while (eps>1e-12):
    yhat, gradient = grad(inp_z, y_meas, W)
    Wnew=W-eta*gradient
    eps=np.sum((W-Wnew)*(W-Wnew))   #eps>1e-12 for this
    #eps=np.sum((yhat-y_meas)*(yhat-y_meas))  #eps>2.5 for this
    W=Wnew
    iter+=1
    if iter%100000==0:
        print(iter, eps)
print(W.T)
fig, ax = plt.subplots()
ax.plot(y_meas, linestyle='None', marker='o')
ax.plot(yhat, color='red')
    